# Setup Libraries and Dependencies

In [1]:
!python --version

Python 3.11.11


## Pip installation

In [2]:
!pip install --disable-pip-version-check -q pip --upgrade > /dev/null
!pip install --disable-pip-version-check -q wrapt --upgrade > /dev/null

## AWS CLI and AWS Python SDK (boto3) Installation

In [3]:
!pip install --disable-pip-version-check -q awscli==1.18.216 boto3==1.16.56 botocore==1.19.56

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.2 requires nvidia-ml-py3==7.352.0, which is not installed.
jupyter-ai 2.29.1 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
aiobotocore 2.20.0 requires botocore<1.36.24,>=1.36.20, but you have botocore 1.19.56 which is incompatible.
amazon-sagemaker-sql-editor 0.1.15 requires boto3<2,>=1.29.6, but you have boto3 1.16.56 which is incompatible.
amazon-sagemaker-sql-editor 0.1.15 requires botocore<2,>=1.31.64, but you have botocore 1.19.56 which is incompatible.
amazon-sagemaker-sql-execution 0.1.6 requires boto3>=1.28.64, but you have boto3 1.16.56 which is incompatible.
amazon-sagemaker-sql-magic 0.1.3 requires sqlparse==0.5.0, but you have sqlparse 0.5.3 which is incompatible.
autogluon-multimodal 1.2 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.23.0 wh

## SageMaker Installation

In [4]:
!pip install --disable-pip-version-check -q sagemaker==2.29.0
!pip install --disable-pip-version-check -q smdebug==1.0.1
!pip install --disable-pip-version-check -q sagemaker-experiments==0.1.26

## PyAthena Installation

In [5]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
amazon-sagemaker-sql-execution 0.1.6 requires boto3>=1.28.64, but you have boto3 1.16.56 which is incompatible.
amazon-sagemaker-sql-execution 0.1.6 requires pyathena<4,>=3.3.0, but you have pyathena 2.1.0 which is incompatible.
amazon-sagemaker-sql-magic 0.1.3 requires sqlparse==0.5.0, but you have sqlparse 0.5.3 which is incompatible.


## AWS Data Wrangler

In [6]:
!pip install --disable-pip-version-check -q awswrangler==3.11.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.2 requires nvidia-ml-py3==7.352.0, which is not installed.
aiobotocore 2.20.0 requires botocore<1.36.24,>=1.36.20, but you have botocore 1.37.34 which is incompatible.
amazon-sagemaker-sql-execution 0.1.6 requires pyathena<4,>=3.3.0, but you have pyathena 2.1.0 which is incompatible.
amazon-sagemaker-sql-magic 0.1.3 requires sqlparse==0.5.0, but you have sqlparse 0.5.3 which is incompatible.
autogluon-multimodal 1.2 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.23.0 which is incompatible.
autogluon-multimodal 1.2 requires nltk<3.9,>=3.4.5, but you have nltk 3.9.1 which is incompatible.
autogluon-multimodal 1.2 requires omegaconf<2.3.0,>=2.1.1, but you have omegaconf 2.3.0 which is incompatible.
awscli 1.18.216 requires botocore==1.19.56, but you have botocore 1.37.34 which is in

## Matplotlib Installation

In [7]:
!pip install --disable-pip-version-check -q matplotlib==3.1.3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.2 requires nvidia-ml-py3==7.352.0, which is not installed.
autogluon-core 1.2 requires matplotlib<3.11,>=3.7.0, but you have matplotlib 3.1.3 which is incompatible.
autogluon-multimodal 1.2 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.23.0 which is incompatible.
autogluon-multimodal 1.2 requires nltk<3.9,>=3.4.5, but you have nltk 3.9.1 which is incompatible.
autogluon-multimodal 1.2 requires omegaconf<2.3.0,>=2.1.1, but you have omegaconf 2.3.0 which is incompatible.
seaborn 0.13.2 requires matplotlib!=3.6.1,>=3.4, but you have matplotlib 3.1.3 which is incompatible.


## Seaborn Installation

In [8]:
!pip install --disable-pip-version-check -q seaborn==0.13.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.2 requires nvidia-ml-py3==7.352.0, which is not installed.
autogluon-multimodal 1.2 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.23.0 which is incompatible.
autogluon-multimodal 1.2 requires nltk<3.9,>=3.4.5, but you have nltk 3.9.1 which is incompatible.
autogluon-multimodal 1.2 requires omegaconf<2.3.0,>=2.1.1, but you have omegaconf 2.3.0 which is incompatible.


## Workcloud installation

In [9]:
!pip install wordcloud

  Using cached wordcloud-1.9.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
Using cached wordcloud-1.9.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (547 kB)


## Sentence Transformers Installation

In [10]:
!pip install -U sentence-transformers

  Using cached sentence_transformers-4.0.2-py3-none-any.whl.metadata (13 kB)
Using cached sentence_transformers-4.0.2-py3-none-any.whl (340 kB)


## Summarize

In [11]:
setup_dependencies_passed = True

In [12]:
%store setup_dependencies_passed

Stored 'setup_dependencies_passed' (bool)


In [13]:
%store

Stored variables and their in-db values:
s3_private_path                         -> 's3://sagemaker-us-east-1-109784353618/linkedin_da
s3_public_path                          -> 's3://linkedin-postings'
setup_dependencies_passed               -> True
setup_iam_roles_passed                  -> True
setup_instance_check_passed             -> True
setup_s3_bucket_passed                  -> True


## Check environment was created correctly

In [14]:
import boto3

region = boto3.Session().region_name
session = boto3.session.Session()

ec2 = boto3.Session().client(service_name="ec2", region_name=region)
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

### _Note: This workshop requires SageMaker Studio and will not work properly in classic SageMaker Notebooks._

In [15]:
import json

notebook_instance_name = None

try:
    with open("/opt/ml/metadata/resource-metadata.json") as notebook_info:
        data = json.load(notebook_info)
        domain_id = data["DomainId"]
        resource_arn = data["ResourceArn"]
        region = resource_arn.split(":")[3]
        name = data["ResourceName"]
    print("DomainId: {}".format(domain_id))
    print("Name: {}".format(name))
except:
    print("+++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR]: COULD NOT RETRIEVE THE METADATA.")
    print("+++++++++++++++++++++++++++++++++++++++++")

DomainId: d-sn9xkgjdn7g8
Name: default


In [16]:
describe_domain_response = sm.describe_domain(DomainId=domain_id)
print(describe_domain_response["Status"])

InService


In [17]:
try:
    get_status_response = sm.get_sagemaker_servicecatalog_portfolio_status()
    print(get_status_response["Status"])
except:
    pass

Enabled


## Summary: Check All Required Settings Are Set Correctly

In [18]:
if (
    describe_domain_response["Status"] == "InService"
    and get_status_response["Status"] == "Enabled"
    and "default" in name
):
    setup_instance_check_passed = True
    print("[OK] Checks passed!  Great Job!!  Please Continue.")
else:
    setup_instance_check_passed = False
    print("+++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR]: WE HAVE IDENTIFIED A MISCONFIGURATION.")
    print(describe_domain_response["Status"])
    print(get_status_response["Status"])
    print(name)
    print("+++++++++++++++++++++++++++++++++++++++++++++++")

[OK] Checks passed!  Great Job!!  Please Continue.


In [19]:
print(setup_instance_check_passed)

True


In [20]:
%store setup_instance_check_passed

Stored 'setup_instance_check_passed' (bool)


In [21]:
%store

Stored variables and their in-db values:
s3_private_path                         -> 's3://sagemaker-us-east-1-109784353618/linkedin_da
s3_public_path                          -> 's3://linkedin-postings'
setup_dependencies_passed               -> True
setup_iam_roles_passed                  -> True
setup_instance_check_passed             -> True
setup_s3_bucket_passed                  -> True


## Create S3 Bucket

In [22]:
import sagemaker

session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

s3 = boto3.Session().client(service_name="s3", region_name=region)

In [23]:
setup_s3_bucket_passed = False

In [24]:
print("Default bucket: {}".format(bucket))

Default bucket: sagemaker-us-east-1-109784353618


# Verify s3_bucket Bucket Creation

In [25]:
%%bash

aws s3 ls s3://${bucket}/

2025-03-18 03:52:12 athena-output-109784353618-default
2025-03-17 01:19:31 aws-athena-query-results-109784353618-us-east-1
2025-03-01 02:05:44 sagemaker-studio-3jnmzv6a1l7
2025-03-01 02:05:46 sagemaker-us-east-1-109784353618


In [26]:
from botocore.client import ClientError

response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(bucket, response, e))

{'ResponseMetadata': {'RequestId': 'J5SNSGPFPETNE36Y', 'HostId': 'mrCGuKc7UVA0BCGH7cI5FVMWClYNUC9ntKMc/02HunYmp2MzcDaj4rBV8t8aKZAr+YI7Y1uveGQ90T0+l44XsaAWrv99AxYA', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'mrCGuKc7UVA0BCGH7cI5FVMWClYNUC9ntKMc/02HunYmp2MzcDaj4rBV8t8aKZAr+YI7Y1uveGQ90T0+l44XsaAWrv99AxYA', 'x-amz-request-id': 'J5SNSGPFPETNE36Y', 'date': 'Tue, 15 Apr 2025 05:28:17 GMT', 'x-amz-bucket-region': 'us-east-1', 'x-amz-access-point-alias': 'false', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'BucketRegion': 'us-east-1', 'AccessPointAlias': False}


In [27]:
%store setup_s3_bucket_passed

Stored 'setup_s3_bucket_passed' (bool)


In [28]:
%store

Stored variables and their in-db values:
s3_private_path                         -> 's3://sagemaker-us-east-1-109784353618/linkedin_da
s3_public_path                          -> 's3://linkedin-postings'
setup_dependencies_passed               -> True
setup_iam_roles_passed                  -> True
setup_instance_check_passed             -> True
setup_s3_bucket_passed                  -> True


## Update IAM Roles and Policies

In [29]:
import time
from time import gmtime, strftime

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

from botocore.config import Config

config = Config(retries={"max_attempts": 10, "mode": "adaptive"})

iam = boto3.client("iam", config=config)

### Get SageMaker Execution Role Name

In [30]:
role_name = role.split("/")[-1]

print("Role name: {}".format(role_name))

Role name: LabRole


In [31]:
setup_iam_roles_passed = False

## Pre-Requisite: SageMaker notebook instance ExecutionRole contains IAMFullAccess Policy.

In [32]:
admin = False
post_policies = iam.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]
for post_policy in post_policies:
    if post_policy["PolicyName"] == "AdministratorAccess":
        admin = True
        break

setup_iam_roles_passed = True
print("[OK] You are all set up to continue with this workshop!")

[OK] You are all set up to continue with this workshop!


In [33]:
%store setup_iam_roles_passed

Stored 'setup_iam_roles_passed' (bool)


In [34]:
%store

Stored variables and their in-db values:
s3_private_path                         -> 's3://sagemaker-us-east-1-109784353618/linkedin_da
s3_public_path                          -> 's3://linkedin-postings'
setup_dependencies_passed               -> True
setup_iam_roles_passed                  -> True
setup_instance_check_passed             -> True
setup_s3_bucket_passed                  -> True


## Release Resources

In [35]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [36]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>